In [1]:
from keras import models, layers
from keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# load the data and devided into 2 sets call train and test data sets 
# training set consists of 60,000 28x28 pixel images, and the test set 10,000.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# initializing the sequential model
model = models.Sequential()
# input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# hidden layers 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# output layer contains 10 neurons for each 0 - 9 digits 
model.add(layers.Dense(10, activation='softmax'))

# compiling the model 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# view model summary which we have created 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [6]:
# adding noise
noise_factor = 0.70
x_train_noisy = train_images + noise_factor * np.random.normal(loc=0.0,scale=1.0,size=train_images.shape)
x_test_noisy = test_images + noise_factor * np.random.normal(loc=0.0,scale=1.0,size=test_images.shape)
x_train_noisy = np.clip(x_train_noisy,0.,1.)
x_test_noisy = np.clip(x_test_noisy,0.,1.)

In [7]:
# reshaping the x_train_noisy images 
x_train_noisy = x_train_noisy.reshape((60000, 28, 28, 1))
x_train_noisy= x_train_noisy.astype('float32') / 255 # rescale pixel values from range [0, 255] to [0, 1]

# reshaping the test images  
x_test_noisy = x_test_noisy.reshape((10000, 28, 28, 1))
x_test_noisy= x_test_noisy.astype('float32') / 255

# label encoding 
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# data partition for model validation
validation_images = x_train_noisy[50000:]
validation_labels = train_labels[50000:]

# data partition for model training 
x_train_noisy = x_train_noisy[:50000]
train_labels = train_labels[:50000]

# train the model
history = model.fit(x_train_noisy, train_labels, epochs=10, batch_size=64, validation_data=(validation_images, validation_labels))

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 1.6310 - accuracy: 0.4169 - val_loss: 0.7397 - val_accuracy: 0.7466
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.5521 - accuracy: 0.8242 - val_loss: 0.4405 - val_accuracy: 0.8614
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3484 - accuracy: 0.8892 - val_loss: 0.2888 - val_accuracy: 0.9060
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2238 - accuracy: 0.9297 - val_loss: 0.1744 - val_accuracy: 0.9455
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1643 - accuracy: 0.9479 - val_loss: 0.1645 - val_accuracy: 0.9495
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1318 - accuracy: 0.9580 - val_loss: 0.1441 - val_accuracy: 0.9576
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1115 - accuracy: 0.9643 - val_loss: 0.1077 - val_accuracy: 0.9663
Epoch 

In [8]:
# validating the model
test_loss, test_acc = model.evaluate(x_test_noisy, test_labels)
print('Accuracy:', test_acc)
print('Loss: ', test_loss)

313/313 [==============================] - 1s 2ms/step - loss: 0.0808 - accuracy: 0.9736
Accuracy: 0.9735999703407288
Loss:  0.08078619092702866


In [9]:
# Creating a second model for removing the noise 
model2 = models.Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
 

model2.add(MaxPooling2D((2, 2), padding='same'))
 
model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model2.add(UpSampling2D((2, 2)))
model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model2.add(UpSampling2D((2, 2)))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(UpSampling2D((2, 2)))
model2.add(Conv2D(1, (3, 3), activation='relu', padding='same'))

model2.compile(optimizer='adam', loss='mean_squared_error')

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 8)         2312      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 8)          

In [10]:
train_images = train_images[:50000]
model2.fit(x_train_noisy, train_images, epochs=10, batch_size=10, shuffle=False, 
          validation_data=(x_test_noisy, test_images))

Epoch 1/10
5000/5000 [==============================] - 14s 3ms/step - loss: 4450.8369 - val_loss: 4439.8174
Epoch 2/10
5000/5000 [==============================] - 14s 3ms/step - loss: 2839.1628 - val_loss: 1826.8170
Epoch 3/10
5000/5000 [==============================] - 14s 3ms/step - loss: 1637.7139 - val_loss: 1522.6404
Epoch 4/10
5000/5000 [==============================] - 14s 3ms/step - loss: 1449.5485 - val_loss: 1395.9709
Epoch 5/10
5000/5000 [==============================] - 14s 3ms/step - loss: 1352.7472 - val_loss: 1328.3043
Epoch 6/10
5000/5000 [==============================] - 14s 3ms/step - loss: 1294.2689 - val_loss: 1279.5168
Epoch 7/10
5000/5000 [==============================] - 14s 3ms/step - loss: 1256.4829 - val_loss: 1247.7379
Epoch 8/10
5000/5000 [==============================] - 14s 3ms/step - loss: 1227.2982 - val_loss: 1227.3625
Epoch 9/10
5000/5000 [==============================] - 15s 3ms/step - loss: 1201.1912 - val_loss: 1197.8361
Epoch 10/10
5000/50

In [11]:
no_noise_img = model2.predict(x_test_noisy)

In [12]:
# evaluating after noise removal
test_loss, test_acc = model.evaluate(no_noise_img, test_labels)
print('Accuracy:', test_acc)
print('Loss: ', test_loss)

313/313 [==============================] - 1s 2ms/step - loss: 8557.9238 - accuracy: 0.8426
Accuracy: 0.8425999879837036
Loss:  8557.923828125
